<p align = "center" draggable=”false” ><img src="https://user-images.githubusercontent.com/37101144/161836199-fdb0219d-0361-4988-bf26-48b0fad160a3.png" 
     width="200px"
     height="auto"/>
</p>

# Reddit and HuggingFace Starter Kit

In this live coding session, we leverage the Python Reddit API Wrapper (`PRAW`) to retrieve data from subreddits on [Reddit](https://www.reddit.com), and perform sentiment analysis using [`pipelines`](https://huggingface.co/docs/transformers/main_classes/pipelines) from [HuggingFace ( 🤗 the GitHub of Machine Learning )](https://techcrunch.com/2022/05/09/hugging-face-reaches-2-billion-valuation-to-build-the-github-of-machine-learning/), powered by [transformer](https://arxiv.org/pdf/1706.03762.pdf).

## Objectives

At the end of the session, you will 

- know how to work with APIs
- feel more comfortable navigating thru documentation, even inspecting the source code
- understand what a `pipeline` object is in HuggingFace
- perform sentiment analysis using `pipeline`
- run a python script in command line and get the results

## How to Submit

- At the end of each task, commit* the work into your own remote repo
- After completing all three tasks, make sure to push the notebook containing all code blocks and output cells to your remote repo
- Submit the link to the notebook in Canvas

\***NEVER** commit a notebook displaying errors unless it is instructed otherwise. However, commit often; recall git ABC = **A**lways **B**e **C**ommitting.

## Tasks

### Task I: Instantiate a Reddit API Object

The first task is to instantiate a Reddit API object using [PRAW](https://praw.readthedocs.io/en/stable/), through which you will retrieve data. PRAW is a wrapper for [Reddit API](https://www.reddit.com/dev/api) that makes interacting with the Reddit API easier unless you are already an expert of [`requests`](https://docs.python-requests.org/en/latest/).

#### 0.  Get updates from `FourthBrain/MLE-7`

Under your forked local repo, **fetch** and download new updates from repo `FourthBrain/MLE-7` locally so you can start development. 
    
<details>
<summary>If you haven't added `FourthBrain/MLE-7` as a remote repo, click here for instructions:</summary>   
You fork repo `FourthBrain/MLE-7` to `yourhandle/MLE-7`, clone it locally, and now you are under directory `MLE-7`. By default, you will see one server name `origin` pointing to your repo:  
    
```
$git remote -v 
origin  git@github.com:yourhandle/MLE-7.git (fetch)
origin  git@github.com:yourhandle/MLE-7.git (push)
```

Think of fetch = read and push = write. 

Now add `FourthBrain/MLE-7` as a remote repo

```
$git add remote fourthbrain git@github.com:FourthBrain/MLE-7.git
$git remote -v
fourthbrain	git@github.com:FourthBrain/MLE-7.git (fetch)
fourthbrain	git@github.com:FourthBrain/MLE-7.git (push)
origin git@github.com:yourhandle/MLE-7.git (fetch)
origin git@github.com:yourhandle/MLE-7.git (push)
```

then before each session starts, run `git fetch fourthbrain` to get updates (why not `git pull`?).

check out [Working with Remotes](https://git-scm.com/book/en/v2/Git-Basics-Working-with-Remotes) for more explanations.
</details>

#### 1. Install packages

```
conda activate {your_virtual_environment_name}
pip install -U transformers praw torch numpy pandas
```

####  2. Create a new app on Reddit 

Create a new app on Reddit and save secret tokens; refer to [post in medium](https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c) for more details.

- Create a Reddit account if you don't have one, log into your account.
- To access the API, we need create an app. Slight updates, on the website, you need to navigate to `preference` > `app`, or click [this link](https://www.reddit.com/prefs/apps) and scroll all the way down. 
- Click to create a new app, fill in the **name**, choose `script`, fill in  **description** and **redirect url** ( The redirect URI is where the user is sent after they've granted OAuth access to your application; more info [here](and details are in [](https://github.com/reddit-archive/reddit/wiki/OAuth2) for our purpose, you can enter some random url, e.g., www.google.com; ) as shown below.
    <img src="https://miro.medium.com/max/700/1*lRBvxpIe8J2nZYJ6ucMgHA.png" width="500"/>
- Jolt down `client_id` (left upper corner) and `client_secret` 

    NOTE: CLIENT_ID refers to 'personal use script" and CLIENT_SECRET to secret.
    <div>
    <img src="https://miro.medium.com/max/700/1*7cGAKth1PMrEf2sHcQWPoA.png" width="300"/>
    </div>

- Create `secrets.py` in the same directory with this notebook, fill in `client_id` and `secret_id` obtained from the last step. We will need to import those constants in the next step.
    ```
    REDDIT_API_CLIENT_ID = {client_id}
    REDDIT_API_CLIENT_SECRET = {secret_id}
    REDDIT_API_USER_AGENT = {can_be_any_string, e.g., : "BotBot"}
    ```
- Add `secrets.py` to your `.gitignore` file if not already done. NEVER push credentials to a repo, private or public. 

In [5]:
!conda create --name py39_12 python=3.9.12 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/cim/anaconda3/envs/py39_12

  added / updated specs:
    - python=3.9.12


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h7f98852_4
  ca-certificates    conda-forge/linux-64::ca-certificates-2021.10.8-ha878542_0
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.36.1-hea4e1c9_2
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5
  libgcc-ng          conda-forge/linux-64::libgcc-ng-12.1.0-h8d9b700_16
  libgomp            conda-forge/linux-64::libgomp-12.1.0-h8d9b700_16
  libnsl             conda-forge/linux-64::libnsl-2.0.0-h7f98852_0
  libuuid            conda-forge/linux-64::libuuid-2.32.1-h7f98852_1000
  libzlib            conda-forge/linux-64::libzlib-1.2.11-h166bdaf_1014
  ncurses        

In [8]:
!conda activate py39_12


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [7]:
!conda init

no change     /home/cim/anaconda3/condabin/conda
no change     /home/cim/anaconda3/bin/conda
no change     /home/cim/anaconda3/bin/conda-env
no change     /home/cim/anaconda3/bin/activate
no change     /home/cim/anaconda3/bin/deactivate
no change     /home/cim/anaconda3/etc/profile.d/conda.sh
no change     /home/cim/anaconda3/etc/fish/conf.d/conda.fish
no change     /home/cim/anaconda3/shell/condabin/Conda.psm1
no change     /home/cim/anaconda3/shell/condabin/conda-hook.ps1
no change     /home/cim/anaconda3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /home/cim/anaconda3/etc/profile.d/conda.csh
no change     /home/cim/.bashrc
No action taken.


#### 3. Instantiate a `Reddit` object

In [ ]:
!pip install -U transformers praw torch numpy pandas

Now you are ready to create a read-only `Reddit` instance. Refer to [documentation](https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html) when necessary.

In [ ]:
import praw
import secrets

# Create a Reddit object which allows us to interact with the Reddit API
reddit = praw.Reddit(
    # YOUR CODE HERE
    # YOUR CODE HERE
    # YOUR CODE HERE
)

In [ ]:
print(reddit) 

<details>
<summary>Expected output:</summary>   

```<praw.reddit.Reddit object at 0x10f8a0ac0>```
</details>

#### 4. Instantiate a `subreddit` object

Lastly, create a `subreddit` object for your favorite subreddit and inspect the object. The expected output you will see ar from `r/machinelearning` unless otherwise specified.

In [ ]:
# YOUR CODE HERE

What is the display name of the subreddit?

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected output:</summary>   

    machinelearning
</details>

How about its title, is it different from the display name?

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected output:</summary>   

    Machine Learning
</details>

Print out the description of the subreddit:

In [ ]:
# YOUR CODE HERE

<details>
<summary>Expected output:</summary>

    **[Rules For Posts](https://www.reddit.com/r/MachineLearning/about/rules/)**
    --------
    +[Research](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AResearch)
    --------
    +[Discussion](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3ADiscussion)
    --------
    +[Project](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict_sr=on&q=flair%3AProject)
    --------
    +[News](https://www.reddit.com/r/MachineLearning/search?sort=new&restrict
</details>

### Task II: Parse comments

#### 1. Top Posts of All Time

Find titles of top 10 posts of **all time** from your favorite subreddit. Refer to [Obtain Submission Instances from a Subreddit Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html)) if necessary. Verify if the titles match what you read on Reddit.

In [ ]:
# try run this line, what do you see? press q once you are done
?subreddit.top 

In [ ]:
# YOUR CODE HERE

<details> <summary>Expected output:</summary>

    [Project] From books to presentations in 10s with AR + ML
    [D] A Demo from 1993 of 32-year-old Yann LeCun showing off the World's first Convolutional Network for Text Recognition
    [R] First Order Motion Model applied to animate paintings
    [N] AI can turn old photos into moving Images / Link is given in the comments - You can also turn your old photo like this
    [D] This AI reveals how much time politicians stare at their phone at work
    [D] Types of Machine Learning Papers
    [D] The machine learning community has a toxicity problem
    [Project] NEW PYTHON PACKAGE: Sync GAN Art to Music with "Lucid Sonic Dreams"! (Link in Comments)
    [P] Using oil portraits and First Order Model to bring the paintings back to life
    [D] Convolution Neural Network Visualization - Made with Unity 3D and lots of Code / source - stefsietz (IG)    
</details>

#### 2. Top 10 Posts of This Week

What are the titles of the top 10 posts of **this week** from your favorite subreddit?

In [ ]:
# YOUR CODE HERE

<details><summary>Expected output:</summary>

    [N] Ian Goodfellow, Apple’s director of machine learning, is leaving the company due to its return to work policy. In a note to staff, he said “I believe strongly that more flexibility would have been the best policy for my team.” He was likely the company’s most cited ML expert.
    [R][P] Thin-Plate Spline Motion Model for Image Animation + Gradio Web Demo
    [P] I’ve been trying to understand the limits of some of the available machine learning models out there. Built an app that lets you try a mix of CLIP from Open AI + Apple’s version of MobileNet, and more directly on your phone's camera roll.
    [R] Meta is releasing a 175B parameter language model
    [N] Hugging Face raised $100M at $2B to double down on community, open-source & ethics
    [P] T-SNE to view and order your Spotify tracks
    [D] : HELP Finding a Book - A book written for Google Engineers about foundational Math to support ML
    [R] Scaled up CLIP-like model (~2B) shows 86% Zero-shot on Imagenet
    [D] Do you use NLTK or Spacy for text preprocessing?
    [D] Democratizing Diffusion Models - LDMs: High-Resolution Image Synthesis with Latent Diffusion Models, a 5-minute paper summary by Casual GAN Papers
</details>

#### 3. Comment Code

Add comments to the code block below to describe what each line of the code does (Refer to [Obtain Comment Instances Section](https://praw.readthedocs.io/en/stable/getting_started/quick_start.html) when necessary). The code is adapted from [this tutorial](https://praw.readthedocs.io/en/stable/tutorials/comments.html)

The purpose is 
1. to understand what the code is doing 
2. start to comment your code whenever it is not self-explantory if you have not (others will thank you, YOU will thank you later 😊) 

In [ ]:
%%time
from praw.models import MoreComments

# YOUR COMMENT HERE
top_comments = []

# YOUR COMMENT HERE
for submission in subreddit.top(limit=10):
    # YOUR COMMENT HERE
    for top_level_comment in submission.comments:
        # YOUR COMMENT HERE
        if isinstance(top_level_comment, MoreComments):
            continue
        # YOUR COMMENT HERE
        top_comments.append(top_level_comment.body)

#### 4. Inspect Comments

How many comments did you extract from the last step? Examine a few comments. 

In [ ]:
#YOUR CODE HERE  # the answer may vary 693 for r/machinelearning

In [ ]:
import random

[random.choice(top_comments) for i in range(3)]

<details> <summary>Some of the comments from `r/machinelearning` subreddit are:</summary>

    ['Awesome visualisation',
    'Similar to a stack or connected neurons.',
    'Will this Turing pass the Turing Test?']
</details>

#### 5. Extract Top Level Comment from Subreddit `TSLA`.

Write your code to extract top level comments from the top 10 topics of a time period, e.g., year, from subreddit `TSLA` and store them in a list `top_comments_tsla`.  

In [ ]:
# YOUR CODE HERE

In [ ]:
len(top_comments_tsla) # Expected: 174 for r/machinelearning

In [ ]:
[random.choice(top_comments_tsla) for i in range(3)]

<details>
<summary>Some of the comments from `r/TSLA` subreddit:</summary>

    ['I bought puts',
    '100%',
    'Yes. And I’m bag holding 1200 calls for Friday and am close to throwing myself out the window']
</details>

### Task III: Sentiment Analysis

Let us analyze the sentiment of comments scraped from `r/TSLA` using a pre-trained HuggingFace model to make the inference. Take a [Quick tour](https://huggingface.co/docs/transformers/quicktour). 

#### 1. Import `pipeline`

In [ ]:
from transformers import # YOUR CODE HERE

#### 2. Create a Pipeline to Perform Task "sentiment-analysis"

In [ ]:
sentiment_model = # YOUR CODE HERE

#### 3. Get one comment from list `top_comments_tsla` from Task II - 5.

In [ ]:
comment = random.choice(top_comments_tsla)

In [ ]:
comment

The example comment is: `'Bury Burry!!!!!'`. Print out what you get. For reproducibility, use the same comment in the next step; consider setting a seed.

#### 4. Make Inference!

In [ ]:
sentiment = # YOUR CODE HERE 

What is the type of the output `sentiment`?

```
YOUR ANSWER HERE
```

In [ ]:
print(f'The comment: {comment}')
print(f'Predicted Label is {sentiment[0]["label"]} and the score is {sentiment[0]["score"]:.3f}')

For the example comment, the output is:

    The comment: Bury Burry!!!!!
    Predicted Label is NEGATIVE and the score is 0.989

### Task IV: Put All Together

Let's pull all the piece together, create a simple script that does 

- get the subreddit
- get comments from the top posts for given subreddit
- run sentiment analysis 

#### Complete the Script

Once you complete the code, running the following block writes the code into a new Python script and saves it as `top_tlsa_comment_sentiment.py` under the same directory with the notebook. 

In [ ]:
%%writefile top_tlsa_comment_sentiment.py

import secrets
import random

from typing import Dict, List

from praw import Reddit
from praw.models.reddit.subreddit import Subreddit
from praw.models import MoreComments

from transformers import pipeline


def get_subreddit(display_name:str) -> Subreddit:
    """Get subreddit object from display name

    Args:
        display_name (str): [description]

    Returns:
        Subreddit: [description]
    """
    reddit = Reddit(
        client_id=secrets.REDDIT_API_CLIENT_ID,        
        client_secret=secrets.REDDIT_API_CLIENT_SECRET,
        user_agent=secrets.REDDIT_API_USER_AGENT
        )
    
    subreddit = # YOUR CODE HERE
    return subreddit

def get_comments(subreddit:Subreddit, limit:int=3) -> List[str]:
    """ Get comments from subreddit

    Args:
        subreddit (Subreddit): [description]
        limit (int, optional): [description]. Defaults to 3.

    Returns:
        List[str]: List of comments
    """
    top_comments = []
    for submission in subreddit.top(limit=limit):
        for top_level_comment in submission.comments:
            if isinstance(top_level_comment, MoreComments):
                continue
            top_comments.append(top_level_comment.body)
    return top_comments

def run_sentiment_analysis(comment:str) -> Dict:
    """Run sentiment analysis on comment using default distilbert model
    
    Args:
        comment (str): [description]
        
    Returns:
        str: Sentiment analysis result
    """
    sentiment_model = # YOUR CODE HERE
    sentiment = sentiment_model(comment)
    return sentiment[0]


if __name__ == '__main__':
    submission = # YOUR CODE HERE
    comments = get_comments(submission)
    comment = # YOUR CODE HERE
    sentiment = run_sentiment_analysis(comment)
    
    print(f'The comment: {comment}')
    print(f'Predicted Label is {sentiment["label"]} and the score is {sentiment["score"]:.3f}')

Run the following block to see the output.

In [ ]:
!python top_tlsa_comment_sentiment.py

<details><summary> Expected output:</summary>

    No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
    The comment: When is DOGE flying
    Predicted Label is POSITIVE and the score is 0.689
</details>